<p><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" align="left" hspace="50px" vspace="0px" style="width:107px;height:152px;"></p>
<h1><font color='#FFFFFFF'> <center>
Proyecto 4</center></font></h1>
<h2><font color='#FFFFFFF'> <center>
Proyecto 2024-02 </center></font></h2>
<h3><font color='#FFFFFFF'> <center>María J. Ostos - Cristian Florez - Juan A. Sañudo</center></font></h3>
<h3><font color='#FFFFFFF'> <center>
2024 </center></font></h3>
<font  face="Courier New" size="3">
<p1><center> </center></p1>

1. Revisión teórica. Del artículo:
https://www.nature.com/articles/s41598-020-59821-7

Hacer un resumen de la sección extracción de caracteríticas (Features extraction) y discutir desde el artículo u otras referencias como se hace cuando desaparecen ciertas formas de ondas en el complejo PQRS debido a alguna enfermedad (10%)

2. Base de datos. Del proyecto pasado hay dos opciones de base de datos, la filtrada por los autores o la filtrada en el proyecto 3. De acuerdo a los resultados del proyecto 3 usar la mejor base de datos y justificar la selección
De la base de datos extraer los registros que correspondan a bradicardia sinusal (SB Sinus Bradycardia) y fibrilación auricular (AFIB Atrial Fibrillation). Esta información está en el archivo Diagnostics.xlsx De estos registros los análisis para el presente proyecto deben hacerse en la derivación II

### Carga de librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.signal import detrend, welch, butter, filtfilt
import pywt
import random
import warnings
import neurokit2 as nk
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
#MLP
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#CNN
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

#LSTM
from tensorflow.keras.layers import LSTM

warnings.filterwarnings("ignore", category=UserWarning)  # Para warnings de tipo UserWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)  # Para warnings de tipo RuntimeWarning

### Carga de datos Diagnostics.xlsx

In [2]:
fs=500

# ruta_diagnostics= r"C:\Users\Lenovo a9 w10\Documents\LabBiosenales\Proyecto_2/Diagnostics.xlsx"
ruta_diagnostics="../Proyecto_3/datos/Diagnostics.xlsx"
df_diagnostics = pd.read_excel(ruta_diagnostics)

# Crear el nuevo dataframe filtrando por la columna 'Rhythm'
datos_diagnostics = df_diagnostics[(df_diagnostics['Rhythm'] == 'SB') | (df_diagnostics['Rhythm'] == 'AFIB')]

#Tamaño del nuevo dataframe
datos_diagnostics.shape

tiempo_diagnostics = np.arange(0, datos_diagnostics.shape[0]/fs,1/fs)

datos_diagnostics.head()

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180113_171327_27000,AFIB,RBBB TWC,85,MALE,117,234,114,356,496,81,-27,19,208,265,386
1,MUSE_20180112_073319_29000,SB,TWC,59,FEMALE,52,52,92,432,401,76,42,8,215,261,431
3,MUSE_20180113_121940_44000,SB,NONE,66,MALE,53,53,96,456,427,34,3,9,219,267,447
5,MUSE_20180112_120347_79000,SB,NONE,46,FEMALE,57,57,70,404,393,38,24,9,225,260,427
6,MUSE_20180114_075026_69000,AFIB,TWC,80,FEMALE,98,86,74,360,459,69,83,17,215,252,395


In [3]:
datos_diagnostics.columns

Index(['FileName', 'Rhythm', 'Beat', 'PatientAge', 'Gender', 'VentricularRate',
       'AtrialRate', 'QRSDuration', 'QTInterval', 'QTCorrected', 'RAxis',
       'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'TOffset'],
      dtype='object')

In [4]:
# Separar nombres de archivo en listas según el ritmo 'SB' y 'AFIB'
sb_files = datos_diagnostics[datos_diagnostics['Rhythm'] == 'SB']['FileName'].tolist()
afib_files = datos_diagnostics[datos_diagnostics['Rhythm'] == 'AFIB']['FileName'].tolist()

In [5]:
def cargar_datos(selected_files,ruta): #Ruta 1 sin filtrar, 2 para filtrados
    # Diccionario para almacenar cada DataFrame y su columna 'II'
    data_frames = {}
    datos_II = {}
    
    # Iterar sobre los archivos en la lista
    for i, file in enumerate(selected_files):
        if ruta==1:
            df = pd.read_csv(f"../Proyecto_3/datos/ECGData/ECGData/{file}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
        elif ruta==2:
            df = pd.read_csv(f"../Proyecto_3/datos/ECGDataDenoised/{file}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
        data_frames[f"df{i+1}"] = df
        # datos_II[f"datos_f{i+1}"] = df["II"]
        datos_II[f"{file}"] = df["II"]
    
    return data_frames, datos_II


3. Herramientas computacionales. Hacer un minitutorial del uso de la herramienta NeuroKit (https://neuropsychology.github.io/NeuroKit/index.html) orientado al análisis de señales ECG (10%)

4. Escoger diferentes señales de la base de datos del punto 2 y aplicar el tutorial del punto 3 (5%).

In [ ]:
## Codigo para pruebas con n sujetos

# Número total de archivos a seleccionar
n_seleccion = 5

# Calcular la mitad para cada ritmo (o una distribución casi mitad y mitad si n_seleccion es impar)
n_sb = n_seleccion // 2
n_afib = n_seleccion - n_sb

# Seleccionar aleatoriamente el número calculado de archivos de cada tipo
selected_sb_files = random.sample(sb_files, min(n_sb, len(sb_files)))
selected_afib_files = random.sample(afib_files, min(n_afib, len(afib_files)))

# Combinar las selecciones de 'SB' y 'AFIB' en la lista final
selected_files = selected_sb_files + selected_afib_files

# Mezclar los archivos seleccionados para no tener un orden específico de 'SB' o 'AFIB'
random.shuffle(selected_files)
# Filtrar el DataFrame para obtener solo las filas de los archivos en selected_files
selected_data = datos_diagnostics[datos_diagnostics['FileName'].isin(selected_files)]

# Crear un diccionario con 'FileName' como clave y 'Rhythm' como valor para fácil acceso
file_rhythm_dict = dict(zip(selected_data['FileName'], selected_data['Rhythm']))

In [ ]:
#Se cargan los datos
all_files = datos_diagnostics['FileName'].tolist()
dfs1,datos_II_ecg=cargar_datos(selected_files,1)
sujetos=datos_II_ecg.keys()




In [14]:
sujetos

dict_keys(['MUSE_20180113_171327_27000', 'MUSE_20180112_073319_29000', 'MUSE_20180113_121940_44000', 'MUSE_20180112_120347_79000', 'MUSE_20180114_075026_69000', 'MUSE_20180114_075128_92000', 'MUSE_20180118_174026_42000', 'MUSE_20180115_125443_25000'])

In [ ]:
# Filtrar el DataFrame para obtener solo las filas de los archivos en selected_files
selected_data_all = datos_diagnostics[datos_diagnostics['FileName'].isin(all_files)]

# Crear un diccionario con 'FileName' como clave y 'Rhythm' como valor para fácil acceso
file_rhythm_all = dict(zip(selected_data_all['FileName'], selected_data_all['Rhythm']))


5. Con las características que extrae el neurokit y las discutidas en el punto 1 generar un conjunto de características para el presente trabajo. Incluir la característica de frecuencia de potencia máxima del Proyecto 3 (10%). No incluir más de 15 características ni menos de 5, justificar la selección (5%).

In [ ]:
#Para todos los datos
all_files = datos_diagnostics['FileName'].tolist()
df,datos_II_ecg=cargar_datos(all_files[:8],1)
sujetos=datos_II_ecg.keys()


In [ ]:
sujetos

In [ ]:
# Filtrar el DataFrame para obtener solo las filas de los archivos en selected_files
selected_data_all = datos_diagnostics[datos_diagnostics['FileName'].isin(all_files)]

# Crear un diccionario con 'FileName' como clave y 'Rhythm' como valor para fácil acceso
file_rhythm_all = dict(zip(selected_data_all['FileName'], selected_data_all['Rhythm']))

df_ECG = pd.DataFrame(columns=["Registro", "Estado","VentricularRate","AtrialRate","QRSCount","QRSDuration","QTInterval","qt_corrected","systole_duration","diastole_duration","systole_diastole_ratio"])


In [ ]:

# for archivo in all_files:
for archivo in sujetos:
    dato=datos_II_ecg[archivo][1:].astype(float)  # Convertir a tipo numérico si es necesario
    nombre="MUSE_20180115_122301_43000"
    estado="SB"
    # Procesar la señal ECG
    Processed_signals, info = nk.ecg_process(dato, sampling_rate=500)

    # Extraer los picos R de la señal ECG y obtener promedio para obtener la frecuencia ventricular
    picos_R = info['ECG_R_Peaks']
    # Calcular los intervalos RR (en segundos) dividiendo por la frecuencia de muestreo
    intervalos_RR = np.diff(picos_R) / fs
    # Calcular la frecuencia ventricular promedio (latidos por minuto)
    VentricularRate  = 60 / np.mean(intervalos_RR)

    # Extraer los picos P de la señal ECG
    picos_P = np.array(info['ECG_P_Peaks'])
    # Filtrar NaN de los picos P para evitar errores de cálculo
    # picos_P_limpios = picos_P[~np.isnan(picos_P)]
    picos_P_limpios = pd.Series(picos_P).interpolate().values #se cambian los NaN interpolandolos. Esto es útil para señales fisiológicas, ya que los datos suelen ser continuos.
    # Calcular la frecuencia auricular promedio (latidos por minuto)
    AtrialRate = 60 / np.mean(np.diff(picos_P_limpios) / 500)

    # Cantidad de complejos QRS
    QRSCount = len(picos_R)  

    # Obtener duración del complejo QRS
    # Obtener el inicio y final del QRS
    Q_onsets = np.array(info["ECG_Q_Peaks"])  # Convertir a numpy array
    Q_offsets = np.array(info["ECG_R_Offsets"])  # Convertir a numpy array
    # Calcular la duración promedio del QRS
    QRS_durations = (Q_offsets - Q_onsets) / fs
    QRSDuration = np.mean(QRS_durations)

    # Intervalo onda QT
    # Obtener el final de la onda T
    T_offsets = info["ECG_T_Offsets"]
    # Calcular el intervalo QT promedio
    QT_intervals = (T_offsets - Q_onsets) / fs
    QTInterval = np.mean(QT_intervals)

    # Calcular QT corregido usando la fórmula de Bazett,
    # El QT corregido (QTc) ajusta el intervalo QT en función de la frecuencia cardíaca.
    qt_corrected = QTInterval / np.sqrt(np.mean(intervalos_RR))
    qt_corrected= np.mean(qt_corrected)

    # Extraer las fases ventriculares (sístole y diástole)
    fases_ventriculares = Processed_signals["ECG_Phase_Ventricular"].values
    # Contar las muestras en cada fase
    conteo_sistole = len(fases_ventriculares[fases_ventriculares == 1])
    conteo_diastole = len(fases_ventriculares[fases_ventriculares == 0])
    # Duración promedio de cada fase
    systole_duration = conteo_sistole / fs  # En segundos
    diastole_duration = conteo_diastole / fs  # En segundos

    # Relación sístole/diástole
    systole_diastole_ratio = systole_duration / diastole_duration

# Obtener el nombre del sujeto (sin extensión .csv)
    nombre_sujeto = archivo[:-4] #Registro

    estado = file_rhythm_all[archivo] #Estado

    df_ECG = pd.concat([df_ECG,
                    pd.DataFrame({'Registro': nombre_sujeto,
                                    'Estado': estado,
                                    "VentricularRate": VentricularRate,
                                    "AtrialRate": AtrialRate,
                                    "QRSCount":QRSCount,
                                    "QRSDuration":QRSDuration,
                                    "QTInterval":QTInterval,
                                    "qt_corrected":qt_corrected,
                                    "systole_duration":systole_duration,
                                    "diastole_duration":diastole_duration,
                                    "systole_diastole_ratio":systole_diastole_ratio},
                                index=[0])
                    ],
                    ignore_index=True)

In [22]:
df_ECG

,Registro,Estado,VentricularRate,AtrialRate,QRSCount,QRSDuration,QTInterval,qt_corrected,systole_duration,diastole_duration,systole_diastole_ratio
0,MUSE_20180113_171327_2,AFIB,117.085863,NaN,19,NaN,NaN,NaN,2.930,6.492,0.451325
1,MUSE_20180112_073319_2,SB,51.698671,51.673228,8,NaN,NaN,NaN,2.978,5.518,0.539688
2,MUSE_20180113_121940_4,SB,53.309640,53.297801,9,NaN,NaN,NaN,3.170,5.836,0.543180
3,MUSE_20180112_120347_7,SB,56.617127,56.590427,9,NaN,NaN,NaN,3.126,5.698,0.548614
4,MUSE_20180114_075026_6,AFIB,95.703956,95.379398,16,NaN,NaN,NaN,4.508,4.898,0.920376
5,MUSE_20180114_075128_9,SB,58.504875,NaN,10,NaN,NaN,NaN,3.218,6.338,0.507731
6,MUSE_20180118_174026_4,SB,58.422590,58.365759,9,NaN,NaN,NaN,3.288,5.294,0.621080
7,MUSE_20180115_125443_2,SB,55.414454,NaN,9,NaN,NaN,NaN,3.524,5.532,0.637021


In [36]:
df_ECG.columns

Index(['Registro', 'Estado', 'VentricularRate', 'AtrialRate', 'QRSCount',
       'QRSDuration', 'QTInterval', 'qt_corrected', 'systole_duration',
       'diastole_duration', 'systole_diastole_ratio'],
      dtype='object')

6. Crear una rutina que aplique sobre todos los archivos de la base de datos la rutina 5 y almacene los resultados en un dataframe:

In [37]:
ruta_base_datos_p3="./datos/Dataframe_P3.xlsx"
df_p3 = pd.read_excel(ruta_base_datos_p3)

In [46]:
datos_diagnostics = datos_diagnostics.rename(columns={'FileName': 'Registro'})

In [ ]:
df_diagnostics_2merge= datos_diagnostics[["Registro","Beat","PatientAge","Gender"]]
df_p3_2merge= df_p3[["Registro","fMP"]]

In [61]:
df_diagnostics_2merge.head()

,Registro,Beat,PatientAge,Gender
0,MUSE_20180113_171327_27000,RBBB TWC,85,MALE
1,MUSE_20180112_073319_29000,TWC,59,FEMALE
3,MUSE_20180113_121940_44000,NONE,66,MALE
5,MUSE_20180112_120347_79000,NONE,46,FEMALE
6,MUSE_20180114_075026_69000,TWC,80,FEMALE


In [62]:
df_p3_2merge.head()

,Registro,fMP
0,MUSE_20180113_171327_2,0.488281
1,MUSE_20180112_073319_2,0.976562
2,MUSE_20180113_121940_4,3.417969
3,MUSE_20180112_120347_7,0.976562
4,MUSE_20180114_075026_6,4.394531


In [ ]:
df_ECG.head()

,Registro,Estado,VentricularRate,AtrialRate,QRSCount,QRSDuration,QTInterval,qt_corrected,systole_duration,diastole_duration,systole_diastole_ratio
0,MUSE_20180113_171327_2,AFIB,117.085863,NaN,19,NaN,NaN,NaN,2.930,6.492,0.451325
1,MUSE_20180112_073319_2,SB,51.698671,51.673228,8,NaN,NaN,NaN,2.978,5.518,0.539688
2,MUSE_20180113_121940_4,SB,53.309640,53.297801,9,NaN,NaN,NaN,3.170,5.836,0.543180
3,MUSE_20180112_120347_7,SB,56.617127,56.590427,9,NaN,NaN,NaN,3.126,5.698,0.548614
4,MUSE_20180114_075026_6,AFIB,95.703956,95.379398,16,NaN,NaN,NaN,4.508,4.898,0.920376
5,MUSE_20180114_075128_9,SB,58.504875,NaN,10,NaN,NaN,NaN,3.218,6.338,0.507731
6,MUSE_20180118_174026_4,SB,58.422590,58.365759,9,NaN,NaN,NaN,3.288,5.294,0.621080
7,MUSE_20180115_125443_2,SB,55.414454,NaN,9,NaN,NaN,NaN,3.524,5.532,0.637021


In [65]:
df_2train=df_ECG.copy()

In [80]:
df_2train = df_2train.merge(df_diagnostics_2merge, on='Registro', how='left')
df_2train = df_2train.merge(df_p3_2merge, on='Registro', how='left')

In [81]:
df_2train

,Registro,Estado,VentricularRate,AtrialRate,QRSCount,QRSDuration,QTInterval,qt_corrected,systole_duration,diastole_duration,systole_diastole_ratio,Beat_x,PatientAge_x,Gender_x,fMP_x,Beat_y,PatientAge_y,Gender_y,fMP_y
0,MUSE_20180113_171327_2,AFIB,117.085863,NaN,19,NaN,NaN,NaN,2.930,6.492,0.451325,NaN,NaN,NaN,0.488281,NaN,NaN,NaN,0.488281
1,MUSE_20180112_073319_2,SB,51.698671,51.673228,8,NaN,NaN,NaN,2.978,5.518,0.539688,NaN,NaN,NaN,0.976562,NaN,NaN,NaN,0.976562
2,MUSE_20180113_121940_4,SB,53.309640,53.297801,9,NaN,NaN,NaN,3.170,5.836,0.543180,NaN,NaN,NaN,3.417969,NaN,NaN,NaN,3.417969
3,MUSE_20180112_120347_7,SB,56.617127,56.590427,9,NaN,NaN,NaN,3.126,5.698,0.548614,NaN,NaN,NaN,0.976562,NaN,NaN,NaN,0.976562
4,MUSE_20180114_075026_6,AFIB,95.703956,95.379398,16,NaN,NaN,NaN,4.508,4.898,0.920376,NaN,NaN,NaN,4.394531,NaN,NaN,NaN,4.394531
5,MUSE_20180114_075128_9,SB,58.504875,NaN,10,NaN,NaN,NaN,3.218,6.338,0.507731,NaN,NaN,NaN,0.976562,NaN,NaN,NaN,0.976562
6,MUSE_20180118_174026_4,SB,58.422590,58.365759,9,NaN,NaN,NaN,3.288,5.294,0.621080,NaN,NaN,NaN,0.976562,NaN,NaN,NaN,0.976562
7,MUSE_20180115_125443_2,SB,55.414454,NaN,9,NaN,NaN,NaN,3.524,5.532,0.637021,NaN,NaN,NaN,0.976562,NaN,NaN,NaN,0.976562


7. Código y análisis de resultados, donde se discutan por los menos tres diferentes arquitecturas de red (10%) y las matrices de confusión obtenidas (10%), de una red neuronal que permita la clasificación de las dos patologías cardiacas.

In [ ]:
df_training = df_2train # Dataframe con todas las columnas y caracteristicas para entrenar
df_training = df_training.iloc[:, :100] #Dataframe de prueba

In [71]:
# Separar la columna objetivo
target_column = 'Estado'
X = df_training.drop(columns=[target_column])  # Datos de entrada
y = df_training[target_column]  # Columna objetivo
num_layers=len(X.columns)

In [72]:
# Preprocesamiento de X
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Pipeline para procesar columnas numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [73]:
# Dividir los datos en entrenamiento y evaluación
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar el preprocesamiento
X_train = preprocessor.fit_transform(X_train)
X_eval = preprocessor.transform(X_eval)

# Convertir la columna objetivo a one-hot encoding
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
y_eval = encoder.transform(y_eval.values.reshape(-1, 1)).toarray()

print("Datos preparados para entrenamiento y evaluación.")


Datos preparados para entrenamiento y evaluación.


15

## Modelos de redes neuronales

1. LSM

In [ ]:
#MLP
def build_mlp(input_shape, output_shape,num_layers):
    model = Sequential([
        Dense(num_layers, activation='relu', input_shape=input_shape),
        Dense(math.ceil(num_layers/2), activation='relu'),
        Dense(output_shape, activation='softmax')  # Salida para clasificación
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# def build_mlp(input_shape, output_shape):
#     model = Sequential([
#         Dense(64, activation='relu', input_shape=input_shape),
#         Dense(32, activation='relu'),
#         Dense(output_shape, activation='softmax')  # Salida para clasificación
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

mlp_model = build_mlp((X_train.shape[1],), y_train.shape[1],num_layers=num_layers)
mlp_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_eval, y_eval))


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.1667 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.8333 - loss: 0.6925 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.8333 - loss: 0.6918 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.8333 - loss: 0.6912 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.8333 - loss: 0.6905 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.8333 - loss: 0.6898 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.8333 - loss: 0.6892 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.8333 - loss: 0.6885 - val_accuracy: 0.5000 - val_loss: 0.

In [79]:
# Evaluar el modelo en el conjunto de evaluación
mpl_loss, mlp_accuracy = mlp_model.evaluate(X_eval, y_eval, verbose=0)
print(f"Exactitud del modelo MLP: {mlp_accuracy * 100:.2f}%")


Exactitud del modelo MLP: 50.00%


2. CNN

In [ ]:

#CNN
def build_cnn(input_shape, output_shape,num_layers):
    model = Sequential([
        Conv1D(num_layers, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(math.ceil(num_layers/2), activation='relu'),
        Dense(output_shape, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# def build_cnn(input_shape, output_shape):
#     model = Sequential([
#         Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
#         MaxPooling1D(pool_size=2),
#         Flatten(),
#         Dense(64, activation='relu'),
#         Dense(output_shape, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

cnn_model = build_cnn((X_train.shape[1], 1), y_train.shape[1],num_layers=num_layers)
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
# cnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_eval, y_eval))

In [ ]:
# Evaluar el modelo en el conjunto de evaluación
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_eval, y_eval, verbose=0)
print(f"Exactitud del modelo CNN: {cnn_accuracy * 100:.2f}%")


3. LSTM

In [ ]:
def build_lstm(input_shape, output_shape,num_layers):
    model = Sequential([
        LSTM(num_layers, activation='tanh', input_shape=input_shape),
        Dense(math.ceil(num_layers/2), activation='relu'),
        Dense(output_shape, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# def build_lstm(input_shape, output_shape):
#     model = Sequential([
#         LSTM(50, activation='tanh', input_shape=input_shape),
#         Dense(64, activation='relu'),
#         Dense(output_shape, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

# Adaptar datos para LSTM (similar a CNN)
lstm_model = build_lstm((X_train.shape[1], 1), y_train.shape[1])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
# lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_eval, y_eval))


In [ ]:
# Evaluar el modelo en el conjunto de evaluación
lstm_loss, lstm_accuracy = cnn_model.evaluate(X_eval, y_eval, verbose=0)
print(f"Exactitud del modelo LSTM: {lstm_accuracy * 100:.2f}%")


8. Consultar cómo funciona, realizar y discutir un ejemplo con los datos, del algoritmo de K means (10%) y máquinas de soporte vectorial (SVM) (10%)

9. Consultar por lo menos tres equipos comerciales traen ya incorporados algoritmos de ayuda diagnóstica a partir de señales EKG (5%) y discutir brevemente las funcionalidades desde la teoría vista en el curso (5%)

## Filtrado de datos 
### (En caso de ser necesario)

In [ ]:
def filtro_wavelet(signal_data, wavelet='db4', nivel=4):
    coeffs = pywt.wavedec(signal_data, wavelet, level=nivel)
    umbral = np.median(np.abs(coeffs[-1])) / 0.6745
    coeffs = [pywt.threshold(c, umbral, mode='soft') for c in coeffs]
    return pywt.waverec(coeffs,wavelet)

# Para cada señal extraer la frecuencia que contiene la máxima potencia y Gráfiquela 
def obtener_frecuencia_maxima_potencia(df, fs=1000):
    nperseg = 1024 
    noverlap = int(nperseg / 2)  
    ventana = np.hamming(nperseg)  
    
    # Calcular la densidad espectral de potencia utilizando Welch
    frecuencias, potencia = welch(df, fs=fs,window=ventana, nperseg=nperseg, noverlap=noverlap)

    # Encontrar la frecuencia con la máxima potencia
    indice_max_potencia = potencia.argmax()
    frecuencia_max = frecuencias[indice_max_potencia]

    return frecuencia_max

# Flujo 2: Detrend, Filtro wavelet, Filtro pasabajas
def flujo_2(data,fs = 1000 ):
    # Paso 1: Detrend
    data_detrended = signal.detrend(data)
    
    # Paso 2: Filtro wavelet
    data_denoised=filtro_wavelet(data_detrended, wavelet='db4', nivel=4)

    # Paso 3: Filtro pasabajas IIR a 50 Hz
    cutoff = 50  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
    data_lowpass = signal.sosfilt(sos, data_denoised)
    
    return data_lowpass

def flujo_2_ajustado(data,fs = 1000 ):
    cutoff= 0.5  # Frecuencia de corte en Hz
    order=4
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    data_f=filtfilt(b, a, data)
    
    # Paso 1: Detrend    
    data_detrended = signal.detrend(data_f)
    
    # Paso 2: Filtro wavelet
    data_denoised=filtro_wavelet(data_detrended, wavelet='db4', nivel=4)

    # Paso 3: Filtro pasabajas IIR a 50 Hz
    cutoff = 50  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
    data_lowpass = signal.sosfilt(sos, data_denoised)
    
    return data_lowpass

In [ ]:
all_files = datos_diagnostics['FileName'].tolist()
df,datos_II=cargar_datos(all_files,1)
sujetos=datos_II.keys()


# Crear un DataFrame vacío para almacenar los resultados
df_resultados_flujo = pd.DataFrame(columns=["Registro", "Estado", "fMP"])

In [ ]:
# Filtrar el DataFrame para obtener solo las filas de los archivos en selected_files
selected_data_all = datos_diagnostics[datos_diagnostics['FileName'].isin(all_files)]

# Crear un diccionario con 'FileName' como clave y 'Rhythm' como valor para fácil acceso
file_rhythm_all = dict(zip(selected_data_all['FileName'], selected_data_all['Rhythm']))

In [ ]:
# for archivo in all_files:
for archivo in sujetos:

    data=datos_II[archivo][1:].astype(float)  # Convertir a tipo numérico si es necesario
    # tiempo = np.arange(0, selected_files[archivo].shape/fs,1/fs)
    tiempo= np.arange(0, data.shape[0]/fs,1/fs)

# Obtener el nombre del sujeto (sin extensión .csv)
    nombre_sujeto = archivo[:-4] #Registro

# Procesar los archivos
    resultados_proceso=flujo_2_ajustado(data,fs)

    estado = file_rhythm_all[archivo] #Estado
    
    F_max=obtener_frecuencia_maxima_potencia(resultados_proceso,fs)
    
    df_resultados_flujo = pd.concat([df_resultados_flujo,
                    pd.DataFrame({'Registro': nombre_sujeto,
                                    'Estado': estado,
                                    'fMP': F_max},
                                index=[0])
                    ],
                    ignore_index=True)

In [ ]:
df_resultados_flujo

In [ ]:
# df_resultados_flujo.to_excel("Dataframe_P3.xlsx", index=False)

## Codigo para probar modelos con datos aleatorios

In [ ]:
# Generar datos simulados
np.random.seed(42)
n_samples = 1000
timesteps = 100  # Longitud de la señal (100 puntos de tiempo)

# Simulación de señales ECG
X = np.random.normal(0, 1, (n_samples, timesteps, 1))  # Señales con ruido
y = np.random.randint(0, 2, n_samples)  # Etiquetas binarias (0: SB, 1: AFIB)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las señales
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, timesteps)).reshape(-1, timesteps, 1)
X_test = scaler.transform(X_test.reshape(-1, timesteps)).reshape(-1, timesteps, 1)

# Convertir etiquetas a formato categórico
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)


# Referencias

[1] 